In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
from preprocessing.LLV.new_LLV import prepare_demand, prepare_hsmdemand, prepare_oh, prepare_demand_daily
# from preprocessing.LLV import preproc_LLV

In [ ]:
from preprocessing.ts_tools import seasonal_plot, plot_periodogram, make_mas, make_lags, make_multistep_target, plot_lags

In [ ]:
#3. define module interface
from dsml_s8e.module import DSMLModule, DSMLModuleRunResult

module = DSMLModule(parameters,run_parameters)

a7s_urls = module.make_artifacts_urls(
    entity_names=[
        "data",
        "oh",
        "consumption",
        "qd",
        "other_skus"])

comp2_urls = module.make_component_resource_urls(
    "0_data_load", env_name = 'prod', 
    entity_names=[
        "consumption",
        "oh",
        "qd"])

# custom_urls = module.make_custom_resource_urls(
#     entity_dict={
#         "entity10":"/data/products/fakeproduct/fakeentity10", 
#         "entity11":"/data/products/fakeproduct2/fakedir/fakeentity11"})

# cache_uls = module.make_cache_urls( 
#     new_cache_entity_names=[
#         "cache_entity1.json", 
#         "cache_entity2"],    
#     last_cache_entity_names=[])

module.print_urls()

In [ ]:
#7. get secrets if needed

from dsml_s8e.security import *

env = module.env_name
stand = module.stand_name

#my_secret_value = get_vault_variable(f'{env}.{stand}.postreSQL.password')

In [ ]:
#5 run spark

from dsml_s8e.spark import SparkEnvironment
SparkEnvironment.stopSparkSession()
spark = SparkEnvironment.runSparkSession()
SparkEnvironment.showSparkUI()
import atexit
_=atexit.register(SparkEnvironment.stopSparkSession)

In [ ]:
SparkEnvironment.showResourceUsage()

In [ ]:
#9. read resources 

cons = spark.read.parquet(comp2_urls.consumption).toPandas()
oh = spark.read.parquet(comp2_urls.oh).toPandas()
qd = spark.read.parquet(comp2_urls.qd).toPandas()

In [ ]:
for df_ in [oh]:
    df_.sort_index(inplace=True)

In [ ]:
import numpy as np
from dateutil import parser
from datetime import datetime
import sys, os
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# len(ddmrp_skus)

# Consumption

In [ ]:
cons = cons.loc[~cons.WEIGHT.isna()]

In [ ]:
cons_daily = prepare_demand_daily(cons.copy())

In [ ]:
cons = prepare_demand(cons.copy())

In [ ]:
cons.groupby([cons.index.year, cons.index.month])['demand'].sum()
# .to_excel('llv.xlsx')

# OH stock

In [ ]:
# def prepare_oh(df):
#     df.columns = ["flag", "sku", "assigned", "not_assigned", "date"]
#     df['booked'] = df[df.flag == 'on stock']['assigned']
#     df['available'] = df[df.flag == 'on stock']['not_assigned']
#     df['in_transit'] = df[df.flag == 'in transit']['assigned'] + df[df.flag == 'in transit']['not_assigned']
#     df['virtual'] = df[df.flag == 'virtual']['assigned'] + df[df.flag == 'virtual']['not_assigned']
#     df["sku"] = df["sku"].apply(lambda x: x.replace("_", "-"))
#     df["booked"] = df["booked"].astype(float)
#     df["available"] = df["available"].astype(float)
#     df = df.fillna(0)
#     df["oh"] = df["booked"] + df["available"]
#     df = df.groupby(by = ['sku', 'date']).sum()[['booked', 'available', 'in_transit', 'virtual', 'oh']].reset_index()
#     return df

In [ ]:
oh

In [ ]:
oh = prepare_oh(oh)

In [ ]:
oh.index = pd.to_datetime(oh.pop('date'), format = '%Y-%m-%d')

# Загрузка данных по QD

In [ ]:
# def prepare_hsmdemand(df, mapping_sku):
#     df.columns = ['sku', 'due_date', 'volume', 'date']
#     df["sku"] = df["sku"].apply(lambda x: x.replace("_", "-"))
#     return df

In [ ]:
qd = prepare_hsmdemand(qd)

In [ ]:
qd.index = pd.to_datetime(qd.pop('date'), format = '%Y-%m-%d')

In [ ]:
qd.volume = qd.volume.astype('int')

In [ ]:
qd

### Вспомогательный df

In [ ]:
week_df = pd.DataFrame(index=pd.date_range('2019', '2024', freq='1d'), columns=['Week of year'])

In [ ]:
week_df['Week of year'] = week_df.index.weekofyear

In [ ]:
week_df.index.name = 'date'

In [ ]:
# cons = cons.reset_index().merge(week_df.reset_index(), how = 'left').set_index('date')
# cons['year'] = cons.index.year

### Сбор датасета

In [ ]:
cons = cons.groupby([cons.index, cons['sku']]).sum().unstack('sku').fillna(0)

In [ ]:
cols = cons.loc[:, 'demand'].columns

In [ ]:
axs = cons.loc(axis=1)[:, cols[7:13]].plot(
    subplots=True, sharex=True, figsize=(11, 9), alpha=0.5,
)
for ax, family in zip(axs, cols[7:13]):
    ax.legend([])
    ax.set_ylabel(family)

In [ ]:
cons = cons.loc[:,'demand'].reset_index().melt(id_vars = 'date', value_name='demand').set_index('date')

In [ ]:
cons['year'] = cons.index.year
cons['month'] = cons.index.month

In [ ]:
plt.figure()
plt.yticks(np.arange(0,1000000, 50000))
sns.lineplot(y = cons.groupby('sku')['demand'].sum().sort_values().quantile(0.5),
                 x = cons.groupby('sku')['demand'].sum().index)
cons.groupby('sku')['demand'].sum().sort_values().plot()
plt.legend()

### Seasonals

In [ ]:
top = list(cons.groupby('sku')['demand'].sum().sort_values(ascending = False).index)[:6]

In [ ]:
fig, axes = plt.subplots(6, 1, figsize = (15,25))
for n, ax in enumerate(fig.get_axes()):
    seasonal_plot(X = cons.loc[cons.sku == top[n], ['month', 'year']],
                  y = cons.loc[cons.sku == top[n], 'demand'], period = 'year', freq = 'month', ax=ax)

#### Periodigram

In [ ]:
plot_periodogram(cons_daily.loc[cons_daily.sku == top[5], 'demand'])

### Lags

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
_ = plot_lags(cons.loc[cons.sku == top[5], 'demand'], lags=6, nrows=2, figsize = (10,7))
_ = plot_pacf(cons.loc[cons.sku == top[5], 'demand'], lags=12)

### Top skus

In [ ]:
# top = (cons.groupby('sku')['demand'].sum()/12).sort_values(ascending = False)

In [ ]:
top = (cons[cons.index > (cons.index[-1] - pd.Timedelta('365d'))].groupby('sku')['demand'].sum()/12).sort_values(ascending = False)

In [ ]:
top = list(top.loc[top.gt(200)].index)

In [ ]:
len(top)

In [ ]:
other_skus = np.setdiff1d(cons.sku.unique(), top)

In [ ]:
len(other_skus)

In [ ]:
ddmrp_skus = top

In [ ]:
cons = cons.loc[cons.sku.isin(ddmrp_skus)]

In [ ]:
cons = cons.sort_values(by = ['date', 'sku'])

In [ ]:
cons.groupby([cons.index.year, cons.index.month])['demand'].sum()

### Lags, MAs, multitarget

In [ ]:
lags = cons.groupby('sku')['demand'].apply(lambda x : make_lags(x, 4))

In [ ]:
mas = cons.groupby('sku')['demand'].apply(lambda x : make_mas(x, np.arange(2,7)))

In [ ]:
ys = cons.groupby('sku')['demand'].apply(lambda x : make_multistep_target(x, 3, 'demand'))

In [ ]:
df = pd.concat([cons, lags, mas, ys], axis = 1)

In [ ]:
target_df = df.dropna(subset=['y_lag_1', 'y_lag_2', 'y_lag_3', 'y_lag_4', 'MA_2', 'MA_3', 'MA_4', 'MA_5', 'MA_6'])

In [ ]:
target_df.loc[target_df.sku == 'DUC031-1450']

### Trend

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess

In [ ]:
y = cons.loc[cons.sku == top[1], 'demand']

In [ ]:
dp = DeterministicProcess(
    index=y.index,
    order=2,
    drop=True,
)

In [ ]:
target_df = target_df.join(dp.in_sample(), how = 'left')

## Подготовка df о планах на потребления

In [ ]:
sl_df = pd.read_excel('Slab_analytic.xlsx', sheet_name= 'PRM')

In [ ]:
mask_base = (sl_df.Division.str.contains('STRIP') & (sl_df.Attribute.str.contains('consumption')))

In [ ]:
sl_df = sl_df.loc[mask_base]

In [ ]:
sl_df = sl_df.sort_values(by = ['PRM data', 'Year'])

In [ ]:
t_df = pd.DataFrame(columns=['year', 'month',  'nam_month', 'pr_m', 'pr_2m'],
                    index = pd.date_range('2020-01', f"{parameters['next_year']}-{parameters['next_month']}", freq='M'))

In [ ]:
t_df['year'] = (t_df.index + pd.Timedelta('10d')).year
t_df['month'] = (t_df.index + pd.Timedelta('10d')).month

In [ ]:
t_df['nam_month'] = (t_df.index + pd.Timedelta('10d')).month_name().str.slice(0,3)

In [ ]:
# t_df['date_of_prediction'] = t_df.index.strftime('%Y%m')

In [ ]:
t_df['full_date_of_prediction'] = t_df.index.strftime('%Y%m%d')

In [ ]:
for f_o_p, year, nam_month in t_df[['full_date_of_prediction', 'year', 'nam_month']].values:
    t_df.loc[(t_df.year == year) & (t_df.nam_month == nam_month), 'pr_m'] =\
        sl_df.loc[(sl_df.Month == nam_month) & (sl_df.Year == year) &\
                          (sl_df['PRM data'].astype('int') <= int(f_o_p)), 'Volume'].values[-1]

In [ ]:
monthes = {'Jan' : '01',
           'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09',
       'Oct':'10', 'Nov':'11', 'Dec':'12'}

In [ ]:
for f_o_p in t_df['full_date_of_prediction'].values:
    max_sl = sl_df.loc[((sl_df.Year.astype('str') +\
    sl_df.Month.apply(lambda x : monthes[x])).astype('int') >\
                        int(f_o_p[:-2])) & (sl_df['PRM data'] <= int(f_o_p))]
    t_df.loc[(t_df['full_date_of_prediction'] == f_o_p), 'pr_2m'] =\
        max_sl.loc[max_sl['PRM data'] == max_sl["PRM data"].astype("int").max(), 'Volume'].iloc[1].mean()

In [ ]:
t_df = t_df.drop(['nam_month', 'full_date_of_prediction'], axis = 1)

## Добавление

In [ ]:
target_df1 = target_df.merge(t_df, how='left')

In [ ]:
target_df1[['pr_m', 'pr_2m']] = target_df1[['pr_m', 'pr_2m']].round(2).astype('float')

In [ ]:
target_df1['grade'] = target_df1.sku.str.extract('(.+)-')

In [ ]:
target_df1['demand'] = target_df1[['demand_1', 'demand_2', 'demand_3']].sum(1)

In [ ]:
# target_df1 = target_df1.dropna()

In [ ]:
target_df1.index.name = 'date'

In [ ]:
target_df1['ew_ma'] = target_df1.groupby('sku')['y_lag_1'].ewm(alpha = 0.8).mean().reset_index().sort_values(['date', 'sku'])['y_lag_1'].values

In [ ]:
target_df1['av_adu'] = target_df1.groupby('sku')['y_lag_1'].expanding().mean().reset_index().sort_values(['date', 'sku'])['y_lag_1'].values

In [ ]:
target_df1['total_MA_6'] = target_df1.groupby(['year', 'month'])['MA_6'].transform('sum')

In [ ]:
target_df1['perc'] = target_df1['MA_6']/target_df1['total_MA_6']

In [ ]:
target_df1['pr_cons'] = target_df1.perc*target_df1.pr_m

In [ ]:
target_df1 = target_df1.where(pd.notnull(target_df1), None)

In [ ]:
#7. make something to create artifacts
df1 = spark.createDataFrame(target_df1.reset_index())
# df2 = spark.createDataFrame(oh.reset_index())
df3 = spark.createDataFrame(cons_daily.reset_index())
# df4 = spark.createDataFrame(qd.reset_index())
# df5 = spark.createDataFrame(qd.reset_index())
# df5 = spark.createDataFrame(in_sup.reset_index())

In [ ]:
#8 write artifacts
df1.write.parquet(a7s_urls.data, mode='overwrite')
# df2.write.parquet(a7s_urls.oh, mode='overwrite')
df3.write.parquet(a7s_urls.consumption, mode='overwrite')
# df4.write.parquet(a7s_urls.qd, mode='overwrite')
# df5.write.parquet(a7s_urls.qd, mode='overwrite')
# df5.write.parquet(a7s_urls.in_sup, mode='overwrite')

In [ ]:
#11 specify result( SUCCESS/FAIL )
module.run_result = DSMLModuleRunResult.SUCCESS

In [ ]:
#10 stop spark

SparkEnvironment.stopSparkSession()